# Import Dependencies

In [1]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine
from config import db_password
from config import db_name

# Read Video Games Data from CSV

In [2]:
# Set CSV file path
path = os.path.join('Output', 'Transformed_video_games_data.csv')

In [3]:
# Read games data from CSV file into Dataframe
transformed_df = pd.read_csv(path)
transformed_df

,rank,game_name,console,publisher,developer,vgchartz_score,critic_score,user_score,total_shipped,global_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,genre,release_year
0,1,God of War,Series,Sony Computer Entertainment,SIE Santa Monica Studio,NaN,NaN,NaN,51.00,NaN,NaN,NaN,NaN,NaN,2005-03-22,Action,2005.0
1,2,Warriors,Series,KOEI,Omega Force,NaN,NaN,NaN,45.26,NaN,NaN,NaN,NaN,NaN,1997-06-30,Action,1997.0
2,3,Devil May Cry,Series,Capcom,Capcom,NaN,NaN,NaN,22.00,NaN,NaN,NaN,NaN,NaN,2001-10-16,Action,2001.0
3,4,Dynasty Warriors,Series,Unknown,Omega Force,NaN,NaN,NaN,21.00,NaN,NaN,NaN,NaN,NaN,NaN,Action,NaN
4,5,Grand Theft Auto V,PS3,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,9.85,0.99,3.12,2013-09-17,Action,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22097,232,"Nora, Princess, and Stray Cat",NS,Harukaze,Harukaze,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN,2018-10-25,Visual Novel,2018.0
22098,233,Memories Off: Innocent File,NS,5pb,5pb. Games,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN,2018-10-25,Visual Novel,2018.0
22099,234,Enkan no Memoria: Kakera Tomoshi,PSV,Dramatic Create,A'sRing,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN,2018-03-29,Visual Novel,2018.0
22100,235,Disorder 6,X360,5pb,5pb. Games,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN,2013-08-22,Visual Novel,2013.0


# Create Genre dataframe

In [4]:
# Generate list of unique genres
genre_list = transformed_df['genre'].unique()
genre_list

array(['Action', 'Action-Adventure', 'Adventure', 'Board Game',
       'Education', 'Fighting', 'Misc', 'MMO', 'Music', 'Party',
       'Platform', 'Puzzle', 'Racing', 'Role-Playing', 'Sandbox',
       'Shooter', 'Simulation', 'Sports', 'Strategy', 'Visual Novel'],
      dtype=object)

In [5]:
# Generate list of genre ids
genre_id_list = list(range(0, len(genre_list)))
genre_id_list

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [6]:
# Create genre dataframe
genre_df = pd.DataFrame({
    'genre_id': genre_id_list,
    'genre': genre_list
})
genre_df

,genre_id,genre
0,0,Action
1,1,Action-Adventure
2,2,Adventure
3,3,Board Game
4,4,Education
5,5,Fighting
6,6,Misc
7,7,MMO
8,8,Music
9,9,Party


# Create Developer dataframe

In [7]:
# Generate list of unique developers
developer_list = transformed_df['developer'].dropna().unique()
developer_list

array(['SIE Santa Monica Studio', 'Omega Force', 'Capcom', ..., 'ADELTA',
       'girls dynamics', "A'sRing"], dtype=object)

In [8]:
# Generate list of developer ids
developer_id_list = list(range(0, len(developer_list)))
developer_id_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [9]:
# Create developer dataframe
developer_df = pd.DataFrame({
    'developer_id': developer_id_list,
    'developer': developer_list
})
developer_df

,developer_id,developer
0,0,SIE Santa Monica Studio
1,1,Omega Force
2,2,Capcom
3,3,Rockstar North
4,4,Konami
...,...,...
3541,3541,Tenco
3542,3542,Gesen 18
3543,3543,ADELTA
3544,3544,girls dynamics


In [10]:
developer_df.dtypes

developer_id     int64
developer       object
dtype: object

# Create Console dataframe

In [11]:
# Generate list of unique consoles
console_list = transformed_df['console'].unique()
console_list

array(['Series', 'PS3', 'PS4', 'PS2', 'X360', 'PC', 'XOne', 'PS', 'PSP',
       'Wii', 'DS', 'All', '3DS', 'NES', 'GC', 'WiiU', 'XB', 'NS', 'N64',
       'GEN', '2600', 'GBA', 'GB', 'PSV', 'SNES', 'DC', 'SAT', 'XBL',
       'PSN', 'GBC', 'PCE', '3DO', 'NG', 'VC', 'WW', 'SCD', 'Mob', 'GG',
       'Amig', 'WS', 'PCFX', 'OSX'], dtype=object)

In [12]:
# Generate list of consoles ids
console_id_list = list(range(0, len(console_list)))
console_id_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41]

In [13]:
# Create console dataframe
console_df = pd.DataFrame({
    'console_id': console_id_list,
    'console': console_list
})
console_df

,console_id,console
0,0,Series
1,1,PS3
2,2,PS4
3,3,PS2
4,4,X360
5,5,PC
6,6,XOne
7,7,PS
8,8,PSP
9,9,Wii


In [14]:
console_df.dtypes

console_id     int64
console       object
dtype: object

# Create Publisher dataframe

In [15]:
# Generate list of unique publishers
publisher_list = transformed_df['publisher'].unique()
publisher_list

array(['Sony Computer Entertainment', 'KOEI', 'Capcom', ..., 'Digiturbo',
       'MAGES', 'Stack'], dtype=object)

In [16]:
# Generate list of publisher ids
publisher_id_list = list(range(0, len(publisher_list)))
publisher_id_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [17]:
# Create publisher dataframe
publisher_df = pd.DataFrame({
    'publisher_id': publisher_id_list,
    'publisher': publisher_list
})
publisher_df

,publisher_id,publisher
0,0,Sony Computer Entertainment
1,1,KOEI
2,2,Capcom
3,3,Unknown
4,4,Rockstar Games
...,...,...
1082,1082,Giza10
1083,1083,Wolfgame
1084,1084,Digiturbo
1085,1085,MAGES


In [18]:
publisher_df.dtypes

publisher_id     int64
publisher       object
dtype: object

# Create Games dataframe

In [19]:
# Generate list of game ids
game_id_list = list(range(0, len(transformed_df)))
game_id_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [20]:
# Generate list of genre ids for games dataframe
genre_id = []
for index, row in transformed_df.iterrows():
    genre_id.append((genre_df.loc[genre_df['genre'] == row[15]]['genre_id']).values[0])
genre_id

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [21]:
# Generate list of console ids for game dataframe
console_id = []
for index, row in transformed_df.iterrows():
    console_id.append((console_df.loc[console_df['console'] == row[2]]['console_id']).values[0])
console_id

[0,
 0,
 0,
 0,
 1,
 0,
 2,
 3,
 2,
 3,
 4,
 3,
 5,
 4,
 2,
 1,
 0,
 1,
 6,
 7,
 8,
 1,
 5,
 3,
 0,
 1,
 1,
 4,
 7,
 7,
 1,
 2,
 9,
 1,
 4,
 1,
 5,
 4,
 1,
 8,
 1,
 7,
 1,
 10,
 4,
 7,
 3,
 7,
 3,
 5,
 11,
 3,
 12,
 1,
 3,
 1,
 4,
 5,
 5,
 7,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 11,
 0,
 2,
 5,
 11,
 3,
 2,
 6,
 3,
 3,
 4,
 4,
 3,
 4,
 7,
 13,
 7,
 5,
 7,
 3,
 2,
 14,
 3,
 3,
 3,
 8,
 0,
 2,
 3,
 7,
 1,
 4,
 15,
 2,
 16,
 11,
 1,
 17,
 11,
 0,
 3,
 11,
 2,
 1,
 5,
 6,
 9,
 4,
 4,
 5,
 7,
 3,
 2,
 10,
 3,
 18,
 19,
 4,
 5,
 1,
 16,
 1,
 7,
 6,
 5,
 2,
 0,
 1,
 7,
 4,
 3,
 2,
 4,
 6,
 7,
 5,
 1,
 11,
 3,
 1,
 3,
 5,
 13,
 1,
 20,
 3,
 5,
 3,
 8,
 21,
 1,
 10,
 4,
 2,
 3,
 12,
 3,
 1,
 9,
 4,
 7,
 4,
 20,
 2,
 1,
 16,
 2,
 4,
 1,
 9,
 5,
 2,
 10,
 5,
 4,
 12,
 3,
 0,
 9,
 7,
 3,
 6,
 5,
 11,
 3,
 3,
 3,
 21,
 4,
 1,
 5,
 2,
 5,
 7,
 22,
 3,
 4,
 3,
 3,
 21,
 10,
 4,
 21,
 1,
 9,
 23,
 14,
 4,
 12,
 1,
 6,
 9,
 5,
 10,
 7,
 4,
 7,
 5,
 3,
 3,
 5,
 1,
 1,
 1,
 11,
 7,
 1,
 4,
 4,
 24,
 11,
 16,

In [22]:
# Generate list of publisher ids for games dataframe
publisher_id = []
for index, row in transformed_df.iterrows():
    publisher_id.append((publisher_df.loc[publisher_df['publisher'] == row[3]]['publisher_id']).values[0])
publisher_id

[0,
 1,
 2,
 3,
 4,
 5,
 4,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 3,
 0,
 4,
 7,
 4,
 0,
 8,
 5,
 1,
 0,
 4,
 4,
 9,
 5,
 5,
 0,
 10,
 11,
 11,
 12,
 13,
 11,
 2,
 4,
 0,
 2,
 11,
 10,
 12,
 14,
 7,
 15,
 0,
 16,
 3,
 17,
 2,
 15,
 0,
 11,
 11,
 18,
 15,
 19,
 11,
 12,
 11,
 3,
 2,
 2,
 6,
 11,
 3,
 3,
 2,
 20,
 3,
 5,
 11,
 11,
 4,
 0,
 15,
 11,
 15,
 2,
 21,
 22,
 2,
 23,
 4,
 17,
 11,
 22,
 10,
 7,
 17,
 0,
 2,
 23,
 14,
 17,
 2,
 21,
 22,
 24,
 11,
 25,
 0,
 22,
 3,
 26,
 15,
 3,
 6,
 11,
 13,
 27,
 17,
 10,
 21,
 28,
 2,
 10,
 29,
 30,
 11,
 22,
 31,
 10,
 32,
 13,
 4,
 21,
 15,
 11,
 12,
 12,
 3,
 0,
 2,
 16,
 21,
 6,
 23,
 11,
 33,
 34,
 10,
 3,
 2,
 16,
 1,
 15,
 35,
 10,
 36,
 2,
 3,
 21,
 5,
 21,
 13,
 37,
 11,
 23,
 11,
 22,
 2,
 0,
 2,
 21,
 17,
 2,
 14,
 38,
 23,
 4,
 23,
 13,
 0,
 10,
 32,
 38,
 37,
 39,
 2,
 2,
 1,
 3,
 10,
 21,
 0,
 11,
 23,
 3,
 40,
 1,
 4,
 21,
 27,
 13,
 24,
 11,
 41,
 42,
 7,
 4,
 13,
 2,
 1,
 22,
 43,
 16,
 7,
 21,
 10,
 0,
 2,
 13,
 22,
 2,
 23,
 3

In [23]:
# Generate list of developer ids for games dataframe
developer_id = []
for index, row in transformed_df.iterrows():
    if str(row[4]) == 'nan':
        developer_id.append(pd.NA)
    else:
        developer_id.append((developer_df.loc[developer_df['developer'] == row[4]]['developer_id']).values[0])
developer_id

[0,
 1,
 2,
 1,
 3,
 4,
 3,
 3,
 5,
 3,
 3,
 6,
 3,
 3,
 0,
 3,
 7,
 5,
 3,
 8,
 9,
 10,
 7,
 11,
 1,
 5,
 12,
 12,
 13,
 11,
 14,
 15,
 16,
 17,
 17,
 18,
 19,
 17,
 2,
 9,
 5,
 2,
 17,
 16,
 18,
 13,
 20,
 21,
 10,
 22,
 2,
 23,
 2,
 18,
 10,
 17,
 17,
 24,
 25,
 26,
 17,
 18,
 17,
 1,
 2,
 2,
 5,
 27,
 2,
 1,
 2,
 28,
 29,
 30,
 31,
 17,
 9,
 32,
 18,
 17,
 16,
 2,
 33,
 34,
 2,
 22,
 6,
 23,
 27,
 35,
 36,
 37,
 38,
 39,
 2,
 40,
 41,
 42,
 2,
 43,
 44,
 14,
 27,
 45,
 10,
 46,
 47,
 48,
 49,
 2,
 47,
 17,
 49,
 2,
 50,
 51,
 43,
 52,
 2,
 16,
 53,
 54,
 27,
 51,
 55,
 16,
 56,
 19,
 3,
 43,
 21,
 17,
 18,
 22,
 57,
 15,
 58,
 22,
 59,
 5,
 16,
 17,
 6,
 60,
 16,
 2,
 58,
 22,
 1,
 18,
 4,
 51,
 4,
 2,
 61,
 16,
 14,
 62,
 49,
 33,
 17,
 40,
 63,
 64,
 65,
 66,
 58,
 43,
 67,
 2,
 68,
 69,
 16,
 3,
 16,
 19,
 66,
 70,
 71,
 72,
 73,
 74,
 75,
 2,
 1,
 76,
 16,
 77,
 32,
 17,
 25,
 57,
 78,
 1,
 79,
 80,
 81,
 25,
 14,
 17,
 82,
 83,
 84,
 3,
 49,
 2,
 1,
 85,
 86,
 43,
 84,
 43,
 7

In [24]:
transformed_df.columns

Index(['rank', 'game_name', 'console', 'publisher', 'developer',
       'vgchartz_score', 'critic_score', 'user_score', 'total_shipped',
       'global_sales', 'na_sales', 'pal_sales', 'japan_sales', 'other_sales',
       'release_date', 'genre', 'release_year'],
      dtype='object')

In [25]:
games_df = pd.DataFrame({
    'game_id' : game_id_list,
    'rank' : transformed_df['rank'],
    'game_name' : transformed_df['game_name'],
    'release_year' : transformed_df['release_year'],
    'release_date' : transformed_df['release_date'],
    'vgchartz_score' : transformed_df['vgchartz_score'],
    'critic_score' : transformed_df['critic_score'],
    'user_score' : transformed_df['user_score'],
    'genre_id' : genre_id,
    'console_id' : console_id,
    'publisher_id' : publisher_id,
    'developer_id' : developer_id
})
games_df

,game_id,rank,game_name,release_year,release_date,vgchartz_score,critic_score,user_score,genre_id,console_id,publisher_id,developer_id
0,0,1,God of War,2005.0,2005-03-22,NaN,NaN,NaN,0,0,0,0
1,1,2,Warriors,1997.0,1997-06-30,NaN,NaN,NaN,0,0,1,1
2,2,3,Devil May Cry,2001.0,2001-10-16,NaN,NaN,NaN,0,0,2,2
3,3,4,Dynasty Warriors,NaN,NaN,NaN,NaN,NaN,0,0,3,1
4,4,5,Grand Theft Auto V,2013.0,2013-09-17,NaN,9.4,NaN,0,1,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...
22097,22097,232,"Nora, Princess, and Stray Cat",2018.0,2018-10-25,NaN,NaN,NaN,19,17,466,1297
22098,22098,233,Memories Off: Innocent File,2018.0,2018-10-25,NaN,NaN,NaN,19,17,270,823
22099,22099,234,Enkan no Memoria: Kakera Tomoshi,2018.0,2018-03-29,NaN,NaN,NaN,19,23,263,3545
22100,22100,235,Disorder 6,2013.0,2013-08-22,NaN,NaN,NaN,19,4,270,823


In [26]:
games_df.dtypes

game_id             int64
rank                int64
game_name          object
release_year      float64
release_date       object
vgchartz_score    float64
critic_score      float64
user_score        float64
genre_id            int64
console_id          int64
publisher_id        int64
developer_id       object
dtype: object

In [27]:
games_df["release_year"] = games_df["release_year"].astype("Int64")
games_df["release_date"] = games_df["release_date"].astype('datetime64[ns]')
games_df["developer_id"] = games_df["developer_id"].astype("Int64")

In [28]:
games_df.dtypes

game_id                    int64
rank                       int64
game_name                 object
release_year               Int64
release_date      datetime64[ns]
vgchartz_score           float64
critic_score             float64
user_score               float64
genre_id                   int64
console_id                 int64
publisher_id               int64
developer_id               Int64
dtype: object

In [29]:
games_df

,game_id,rank,game_name,release_year,release_date,vgchartz_score,critic_score,user_score,genre_id,console_id,publisher_id,developer_id
0,0,1,God of War,2005,2005-03-22,NaN,NaN,NaN,0,0,0,0
1,1,2,Warriors,1997,1997-06-30,NaN,NaN,NaN,0,0,1,1
2,2,3,Devil May Cry,2001,2001-10-16,NaN,NaN,NaN,0,0,2,2
3,3,4,Dynasty Warriors,<NA>,NaT,NaN,NaN,NaN,0,0,3,1
4,4,5,Grand Theft Auto V,2013,2013-09-17,NaN,9.4,NaN,0,1,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...
22097,22097,232,"Nora, Princess, and Stray Cat",2018,2018-10-25,NaN,NaN,NaN,19,17,466,1297
22098,22098,233,Memories Off: Innocent File,2018,2018-10-25,NaN,NaN,NaN,19,17,270,823
22099,22099,234,Enkan no Memoria: Kakera Tomoshi,2018,2018-03-29,NaN,NaN,NaN,19,23,263,3545
22100,22100,235,Disorder 6,2013,2013-08-22,NaN,NaN,NaN,19,4,270,823


# Create Sales dataframe

In [40]:
sales_df = pd.DataFrame({
    'game_id': game_id_list,
    'na_sales_in_millions': transformed_df['na_sales'],
    'pal_sales_in_millions': transformed_df['pal_sales'],
    'japan_sales_in_millions': transformed_df['japan_sales'],
    'other_sales_in_millions': transformed_df['other_sales'],
    'global_sales_in_millions': transformed_df['global_sales'],
    'total_shipped_in_millions': transformed_df['total_shipped']
})
sales_df

,game_id,na_sales_in_millions,pal_sales_in_millions,japan_sales_in_millions,other_sales_in_millions,global_sales_in_millions,total_shipped_in_millions
0,0,NaN,NaN,NaN,NaN,NaN,51.00
1,1,NaN,NaN,NaN,NaN,NaN,45.26
2,2,NaN,NaN,NaN,NaN,NaN,22.00
3,3,NaN,NaN,NaN,NaN,NaN,21.00
4,4,6.37,9.85,0.99,3.12,20.32,NaN
...,...,...,...,...,...,...,...
22097,22097,NaN,NaN,0.00,NaN,0.00,NaN
22098,22098,NaN,NaN,0.00,NaN,0.00,NaN
22099,22099,NaN,NaN,0.00,NaN,0.00,NaN
22100,22100,NaN,NaN,0.00,NaN,0.00,NaN


In [41]:
sales_df.dtypes

game_id                        int64
na_sales_in_millions         float64
pal_sales_in_millions        float64
japan_sales_in_millions      float64
other_sales_in_millions      float64
global_sales_in_millions     float64
total_shipped_in_millions    float64
dtype: object

# Load Genre Dataframe into PostgreSQL

In [42]:
param_dic = {
    "host"      : "localhost",
    "database"  : db_name,
    "user"      : "postgres",
    "password"  : db_password
}

try:
    connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
        param_dic['user'],
        param_dic['password'],
        param_dic['host'],
        param_dic['database']
    )

    engine = create_engine(connect)
    print("Connection established...")
    
    # Write data from Dataframe into Database Tables
    genre_df.to_sql(
        'genre', 
        con=engine,
        if_exists='append',
        index=False
    )
    print("Inserted data from Genre Dataframe into genre table...")

except:
    print("Error while connecting/writing/reading PostgreSQL")

Connection established...
Inserted data from Genre Dataframe into genre table...


# Load Console Dataframe into PostgreSQL

In [43]:
param_dic = {
    "host"      : "localhost",
    "database"  : db_name,
    "user"      : "postgres",
    "password"  : db_password
}

try:
    connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
        param_dic['user'],
        param_dic['password'],
        param_dic['host'],
        param_dic['database']
    )

    engine = create_engine(connect)
    print("Connection established...")
    
    # Write data from Dataframe into Database Tables
    console_df.to_sql(
        'console', 
        con=engine,
        if_exists='append',
        index=False
    )
    print("Inserted data from Console Dataframe into console table...")

except:
    print("Error while connecting/writing/reading PostgreSQL")

Connection established...
Inserted data from Console Dataframe into console table...


# Load Publisher Dataframe into PostgreSQL

In [44]:
param_dic = {
    "host"      : "localhost",
    "database"  : db_name,
    "user"      : "postgres",
    "password"  : db_password
}

try:
    connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
        param_dic['user'],
        param_dic['password'],
        param_dic['host'],
        param_dic['database']
    )

    engine = create_engine(connect)
    print("Connection established...")
    
    # Write data from Dataframe into Database Tables
    publisher_df.to_sql(
        'publisher', 
        con=engine,
        if_exists='append',
        index=False
    )
    print("Inserted data from publisher Dataframe into publisher table...")

except:
    print("Error while connecting/writing/reading PostgreSQL")

Connection established...
Inserted data from publisher Dataframe into publisher table...


# Load Developer Dataframe into PostreSQL

In [45]:
param_dic = {
    "host"      : "localhost",
    "database"  : db_name,
    "user"      : "postgres",
    "password"  : db_password
}

try:
    connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
        param_dic['user'],
        param_dic['password'],
        param_dic['host'],
        param_dic['database']
    )

    engine = create_engine(connect)
    print("Connection established...")
    
    # Write data from Dataframe into Database Tables
    developer_df.to_sql(
        'developer', 
        con=engine,
        if_exists='append',
        index=False
    )
    print("Inserted data from developer Dataframe into developer table...")

except:
    print("Error while connecting/writing/reading PostgreSQL")

Connection established...
Inserted data from developer Dataframe into developer table...


# Write Games Dataframe Into PostgreSQL

In [46]:
param_dic = {
    "host"      : "localhost",
    "database"  : db_name,
    "user"      : "postgres",
    "password"  : db_password
}

try:
    connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
        param_dic['user'],
        param_dic['password'],
        param_dic['host'],
        param_dic['database']
    )

    engine = create_engine(connect)
    print("Connection established...")
    
    # Write data from Dataframe into Database Tables
    games_df.to_sql(
        'games', 
        con=engine,
        if_exists='append',
        index=False
    )
    print("Inserted data from games Dataframe into games table...")

except:
    print("Error while connecting/writing/reading PostgreSQL")

Connection established...
Inserted data from games Dataframe into games table...


# Write Sales Dataframe into PostgreSQL

In [47]:
param_dic = {
    "host"      : "localhost",
    "database"  : db_name,
    "user"      : "postgres",
    "password"  : db_password
}

try:
    connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
        param_dic['user'],
        param_dic['password'],
        param_dic['host'],
        param_dic['database']
    )

    engine = create_engine(connect)
    print("Connection established...")
    
    # Write data from Dataframe into Database Tables
    sales_df.to_sql(
        'sales', 
        con=engine,
        if_exists='append',
        index=False
    )
    print("Inserted data from sales Dataframe into sales table...")

except:
    print("Error while connecting/writing/reading PostgreSQL")

Connection established...
Inserted data from sales Dataframe into sales table...


# Write the Dataframes into CSV files

In [48]:
genre_df.to_csv('Output/genre_data.csv', index=False)
console_df.to_csv('Output/console_data.csv', index=False)
publisher_df.to_csv('Output/publisher_data.csv', index=False)
developer_df.to_csv('Output/developer_data.csv', index=False)
games_df.to_csv('Output/games_data.csv', index=False)
sales_df.to_csv('Output/sales_data.csv', index=False)